# Use ONNX Bidirectional Attention Flow


 In this notebook, we investigate how to use the [BiDAF](https://arxiv.org/abs/1611.01603) natural language processing model serialized in [ONNX](https://onnx.ai/) format. For performing this experiment, we will use some random contexts and queries.

## Setup Environment

But before we start, let's set up our working environment. It will help us to keep the project directory clean. All artifacts generated within the project will be placed to the "tmp" dir and will be ignored by Git.

In [2]:
# Let's make a temporary directory for all artifacts created by this notebook.
import os
if not os.path.exists('tmp'):
    os.makedirs('tmp')

## Download ResNet ONNX Model

Use the following [page](https://github.com/onnx/models/tree/master/text/machine_comprehension/bidirectional_attention_flow) to get the URL for the BiDAF ONNX model.

**Please note that downloading ONNX models have a large size. To prevent the repeated download of the same model on the notebook restart we check whether the model already exists. If a model exists we skip the download. To force download, go to the "tmp" directory and delete the "bidaf.onnx" file.**

In [3]:
if not os.path.exists(os.path.join('tmp', 'bidaf.onnx')):
    !wget -O tmp/resnet.onnx https://github.com/onnx/models/raw/master/text/machine_comprehension/bidirectional_attention_flow/model/bidaf-9.onnx 
                      

--2021-09-14 13:10:18--  https://github.com/onnx/models/raw/master/text/machine_comprehension/bidirectional_attention_flow/model/bidaf-9.onnx
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/onnx/models/master/text/machine_comprehension/bidirectional_attention_flow/model/bidaf-9.onnx [following]
--2021-09-14 13:10:18--  https://media.githubusercontent.com/media/onnx/models/master/text/machine_comprehension/bidirectional_attention_flow/model/bidaf-9.onnx
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43522228 (42M) [application/octet-stream]
Saving to: ‘tmp/resnet.onnx’

tmp/resne

## Define Context/Query Preprocessor

Before using BiDAF, we need to preprocess the context/query using the predefined function.

In [7]:
import numpy as np
from nltk import word_tokenize

import nltk
nltk.download('punkt')

def preprocess(text):
   tokens = word_tokenize(text)
   # split into lower-case word tokens, in numpy array with shape of (seq, 1)
   words = np.asarray([w.lower() for w in tokens]).reshape(-1, 1)
   # split words into chars, in numpy array with shape of (seq, 1, 1, 16)
   chars = [[c for c in t][:16] for t in tokens]
   chars = [cs+['']*(16-len(cs)) for cs in chars]
   chars = np.asarray(chars).reshape(-1, 1, 1, 16)
   return words, chars

[nltk_data] Downloading package punkt to /home/mykola/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Generate Answer

First we need to load the model.

In [5]:
import onnxruntime as rt

sess = rt.InferenceSession(os.path.join('tmp', 'bidaf.onnx'))

2021-09-14 12:15:56.811330155 [W:onnxruntime:, graph.cc:1074 Graph] Initializer Word_Embedding appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2021-09-14 12:15:56.811350563 [W:onnxruntime:, graph.cc:1074 Graph] Initializer Char_Embedding appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2021-09-14 12:15:56.811354731 [W:onnxruntime:, graph.cc:1074 Graph] Initializer __OneFloat appears in graph inpu

Create input (context and query).

In [78]:
context = '''
Specifically, the hottest spot ever recorded on Earth is El Azizia, in Libya,
where a temperature of 136 degrees Fahrenheit was recorded on Sept. 13, 1922.
While hotter spots have likely occurred in other parts of the planet at other
times, this is the most scorching temperature ever formally recorded by a
weather station. 
'''
query = 'By which station it was recorded?'
cw, cc = preprocess(context)
qw, qc = preprocess(query)

Generate answer.

In [79]:
answer = sess.run([], {'context_word':cw, 'context_char':cc, 'query_word':qw, 'query_char':qc})
answer

[array([58], dtype=int32), array([59], dtype=int32)]

In [80]:
# assuming answer contains the np arrays for start_pos/end_pos
starts, ends = answer
for start, end in zip(starts, ends):
    s = np.asscalar(start)
    e = np.asscalar(end)
    print([w for w in cw[s:e+1].reshape(-1)])

['weather', 'station']


/tmp/ipykernel_11145/1183086202.py:4: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  s = np.asscalar(start)
/tmp/ipykernel_11145/1183086202.py:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  e = np.asscalar(end)
